In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'NanumGothicCoding'
plt.rcParams['font.size'] = 10
import seaborn as sns

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

앙상블(Ensemble)

지도 학습은 피쳐 데이터와 레이블 데이터를 이용해서 전체 데이터를 분류하는 학습 방법이다.  
지도 학습에 사용되는 데이터는 각 피쳐 데이터 마다 올바르게 할당된 라벨링 데이터였고 이를 기반으로 학습 알고리즘을 생성하는 과정을 거쳤었다.

앙상블 학습의 핵심 아이디어는 학습 데이터를 기반으로 분류 모델을 여러개 만들고 서로 비교하는 것이다.  
앙상블 학습 과정에서 만든 개별 분류기 모형을 분류기(Classifier)라고 하고 여러 개의 분류기를 결합함으로써 개별적인 분류기보다 성능이 뛰어난 최종 분류기를 만드는 것이 앙상블 학습의 목적이다.

보팅(Voting)

여러 개의 분류 모델의 결과를 대상으로 투표를 통해서 최종 클래스에 레이블을 결정하는 방법이다.

분류기가 10개 있다고 했을 때, 특정 데이터에 대해서 7개의 분류기는 클래스1 이라고 예측하고, 나머지 3개의 분류기는 클래스2 라고 예측했을 때 클래스1이 가장 높은 득표수를 보이므로 최종적으로 클래스1로 예측하는 것이다. 이를 다수결 투표라고 하는데 이와 비슷한 방법으로 다수결이 아닌 절반 이상의 분류기의 표를 얻어야 하는 과반수 투표 방식이 있다.

개별 분류기는 지도 학습 방법 중 k-최근접 이웃, 로지스틱 회귀, 나이브 베이즈, 의사결정 트리, 서포트 벡터 머신 등 여러가지 알고리즘을 사용해서 다양한 분류 모델을 만들어 사용할 수 있다.

붓꽃 데이터를 사용해 붓꽃 종류를 분류하는 모델을 생성하고 학습시킨다.

In [2]:
# 데이터 불러오기
raw_data = datasets.load_iris() # 사이킷런이 제공하는 붓꽃 데이터를 불러온다.

# 피쳐, 레이블 데이터 저장
xData = raw_data.data # 피쳐 데이터를 저장한다.
yData = raw_data.target # 피쳐 데이터에 따른 레이블을 저장한다.
print(xData.shape, yData.shape)

# 학습 데이터와 테스트 데이터로 분할
x_train, x_test, y_train, y_test = train_test_split(xData, yData, random_state=0)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

# 데이터 표준화
std_scale = StandardScaler() # 표준화 스케일러 객체를 만든다.
x_train = std_scale.fit_transform(x_train) # 학습 데이터를 스케일러로 표준화 하고 적용한다.
x_test = std_scale.transform(x_test) # 테스트 데이터를 학습 데이터로 표준화 스케일러에 적용한다.

(150, 4) (150,)
(112, 4) (38, 4) (112,) (38,)


모델 생성 후 데이터 학습

In [3]:
from sklearn.linear_model import LogisticRegression # 로지스틱 회귀 알고리즘을 사용하기 위해 import 한다.
from sklearn.svm import SVC # 서포트 벡터 머신 알고리즘을 사용하기 위해 import 한다.
from sklearn.naive_bayes import GaussianNB # 가우시안 나이브 베이즈 알고리즘을 사용하기 위해 import 한다.
from sklearn.ensemble import VotingClassifier # 앙상블 보팅 알고리즘을 사용하기 위해 import 한다.

clf1 = LogisticRegression() # 로지스틱 회귀 알고리즘을 사용하는 개별 분류기를 만든다.
clf2 = SVC(probability=True) # 서포트 벡터 머신 알고리즘을 사용하는 개별 분류기를 만든다.
clf3 = GaussianNB() # 가우시안 나이브 베이즈 알고리즘을 사용하는 개별 분류기를 만든다.

# 위 3개의 개별 분류기를 이용해서 앙상블 보팅 모델을 만든다.
# estimators 속성으로 앙상블 보팅 모델이 사용할 개별 분류기를 지정한다.
# voting 속성으로 투표 방식을 지정한다. 'hard'는 기본값으로 과반수 투표를 'soft'는 다수결 투표를 한다.
# soft 보팅을 사용할 경우 개별 분류기를 구성하는 서포트 벡터 머신 모델이 있다면 probability=True 속성을
# 반드시 지정해야 에러가 발생되지 않는다.
# weights 속성으로 개별 분류기에 가중치를 지정해서 가중치 투표를 할 수 있다.
clf = VotingClassifier(estimators=[('lr', clf1), ('svm', clf2), ('gnb', clf3)], voting='hard', weights=[1, 1, 1])
# 표준화된 학습 데이터와 학습 데이터에 따른 레이블 데이터로 앙상블 보팅 모델을 학습시킨다.
clf.fit(x_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('gnb', GaussianNB())],
                 weights=[1, 1, 1])

학습된 모델로 테스트 데이터를 예측한다.

In [4]:
# predict() 메소드의 인수로 표준화된 테스트 데이터를 넘겨서 예측한다.
predict = clf.predict(x_test)
print(predict)

[2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0 2 1 0 2 2 1 0
 2]


학습된 모델을 평가한다.

In [5]:
# 혼동 행렬
# confusion_matrix() 메소드의 인수를 테스트 데이터의 실제값, 예측값 순서로 넘겨서 혼동 행렬를 만든다.
conf_matrix = confusion_matrix(y_test, predict)
print(conf_matrix)

[[13  0  0]
 [ 0 15  1]
 [ 0  0  9]]


In [6]:
# 분류 리포트
# classification_report() 메소드의 인수를 테스트 데이터의 실제값, 예측값 순서로 넘겨서 분류 리포트를 만든다.
# target_names 속성으로 분류 리포트에 레이블의 실제값을 출력할 수 있다.
class_report = classification_report(y_test, predict, target_names=raw_data.target_names) 
print(class_report)

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        13
  versicolor       1.00      0.94      0.97        16
   virginica       0.90      1.00      0.95         9

    accuracy                           0.97        38
   macro avg       0.97      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38

